In [1]:
import numpy as np
import pandas as pd
import sys
import pickle

sys.path.append('../')
sys.path.append('../utils/')

from utils.models import Predictor
from utils.common_utils import *
from utils.data_prepare_utils import *
from utils.bayes_utils import *
from utils.process_finder import *

Using TensorFlow backend.


In [2]:
file_path = '../data/train_data/okaribe_train.csv'
weight_path = '../model/model_weights/okaribe_train_cls_high_acc.h5'

In [3]:
predictor = Predictor(file_path)
predictor.load_weight(weight_path)

/home/rewu/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 766 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 25154 validated image filenames belonging to 8 classes.


In [ ]:
df_cls_res = get_classifier_result(file_path,predictor)



21239/25154 [========================>.....] - ETA: 44s

# Combine detect, Cls, original 

In [ ]:
yolo_train_res_path = '/home/rewu/Documents/research/cv_tunnel/train-master/result_okaribe/detect_train.csv'

df_yolo_veh = pd.read_csv(yolo_train_res_path)
df_full_res_clean = pd.read_csv(file_path)

In [ ]:
df_cls_obj_pred = combine_detect_cls(df_cls_res,df_yolo_veh)

In [ ]:
df_cls_obj_pred.head()

In [ ]:
df_tot = pd.merge(df_cls_obj_pred,df_full_res_clean,on='path',how='inner')

In [ ]:
start_date = '2019-01-01 00:00:00'
end_date = '2019-04-10 00:00:00'
short_period_df= select_short_period_df(df_tot,start_date,end_date)



In [ ]:
df_time,start_end_time_pairs = get_df_with_process(short_period_df)

# Bayesian Prediction

In [ ]:
time_prob_df = pd.read_csv('data/okaribe_time_cond_prob.csv')
veh_prob_df = pd.read_csv('data/veh_info.csv')

with open('data/vehicle_names.data', 'rb') as filehandle:
    veh_names = pickle.load(filehandle)
    
prior_knowledge = (time_prob_df,veh_prob_df,veh_names)

In [ ]:
df_time,anom_scores = update_combined_res(df_time,start_end_time_pairs,prior_knowledge)